Requirements

In [2]:
required_desc = [
'alcoholic', 'aldehydic', 'alliaceous', 'almond', 'amber', 'animal',
'anisic', 'apple', 'apricot', 'aromatic', 'balsamic', 'banana', 'beefy',
'bergamot', 'berry', 'bitter', 'black currant', 'brandy', 'burnt',
'buttery', 'cabbage', 'camphoreous', 'caramellic', 'cedar', 'celery',
'chamomile', 'cheesy', 'cherry', 'chocolate', 'cinnamon', 'citrus', 'clean',
'clove', 'cocoa', 'coconut', 'coffee', 'cognac', 'cooked', 'cooling',
'cortex', 'coumarinic', 'creamy', 'cucumber', 'dairy', 'dry', 'earthy',
'ethereal', 'fatty', 'fermented', 'fishy', 'floral', 'fresh', 'fruit skin',
'fruity', 'garlic', 'gassy', 'geranium', 'grape', 'grapefruit', 'grassy',
'green', 'hawthorn', 'hay', 'hazelnut', 'herbal', 'honey', 'hyacinth',
'jasmin', 'juicy', 'ketonic', 'lactonic', 'lavender', 'leafy', 'leathery',
'lemon', 'lily', 'malty', 'meaty', 'medicinal', 'melon', 'metallic',
'milky', 'mint', 'muguet', 'mushroom', 'musk', 'musty', 'natural', 'nutty',
'odorless', 'oily', 'onion', 'orange', 'orangeflower', 'orris', 'ozone',
'peach', 'pear', 'phenolic', 'pine', 'pineapple', 'plum', 'popcorn',
'potato', 'powdery', 'pungent', 'radish', 'raspberry', 'ripe', 'roasted',
'rose', 'rummy', 'sandalwood', 'savory', 'sharp', 'smoky', 'soapy',
'solvent', 'sour', 'spicy', 'strawberry', 'sulfurous', 'sweaty', 'sweet',
'tea', 'terpenic', 'tobacco', 'tomato', 'tropical', 'vanilla', 'vegetable',
'vetiver', 'violet', 'warm', 'waxy', 'weedy', 'winey', 'woody'
]

In [3]:
len(required_desc)

138

analysis

In [4]:
import pyrfume
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

/home/aryanab/anaconda3/envs/deepchem/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[raw]
"leffingwell_data.csv" = "Odor labels for molecules as originally compiled by John Leffingwell and cleaned by Sanchez-Lengeling et al"
"leffingwell_readme.pdf" = "Information about the dataset"
LICENSE = "Licensing information and use restrictions according to the terms of John Leffingwell and Google"

[processed]
"molecules.csv" = "Information about odorant molecules used"
"behavior.csv" = "Odor labels for each molecule (one column per label)"
"behavior_sparse.csv" = "Odor labels for each molecule (single column)"
"stimuli.csv" = "Maps stimulus to CID, negative numbers assigned to compounds without CIDs"

In [5]:
molecules = pyrfume.load_data('leffingwell/molecules.csv', remote=True)
behavior = pyrfume.load_data('leffingwell/behavior.csv', remote=True)
behavior_sparse = pyrfume.load_data('leffingwell/behavior_sparse.csv', remote=True)

possible tests: molecules
- duplicate smiles?
- convert all smiles to canonical smiles then check
- check if all are valid smiles

In [6]:
molecules.head()

,MolecularWeight,IsomericSMILES,IUPACName,name
CID,,,,
-955348933095,240.387,CCCCC=COC(=O)CCCCCCCC,NaN,Hexenyl nonanoate
-923209957509,196.290,CC(=O)OCC1C=CC(C(C)C)CC1,NaN,Tetrahydrocuminyl acetate
-874408321546,244.331,CCCCCCCCC(OC(C)=O)C(=O)OC,NaN,Methyl acetoxydecanoate
-873963935677,198.306,CCCCC=COC(=O)C(C)CCC,NaN,Hexenyl methylvalerate
-862841422647,148.271,CCCC(S)COCC,NaN,Ethoxymethylbutanethiol


find '1-methyl-3-hexenyl acetate' aka => '4-hepten-2-ol, (e)-, acetate'

In [7]:
from rdkit import Chem
from rdkit.Chem import Draw
import matplotlib.pyplot as plt

def mol_img(smiles, name):
    m = Chem.MolFromSmiles(smiles)
    img = Draw.MolToImage(m)
    # img.title(smiles)
    # img.show(title = smiles)
    with open(f"imgs/'{name}'.png", 'wb')as fp:
        img.save(fp)

In [8]:
mol_img('CCC=CCC(C)OC(C)=O', '1-methyl-3-hexenyl acetate')

In [9]:
molecules[molecules['IsomericSMILES']=='CC/C=C/CC(C)OC(=O)C']

,MolecularWeight,IsomericSMILES,IUPACName,name
CID,,,,
5352813,156.22,CC/C=C/CC(C)OC(=O)C,[(E)-hept-4-en-2-yl] acetate,"4-hepten-2-ol, (e)-, acetate"


In [10]:
k = molecules[molecules['name'].str.match('.*acetate.*')]
l = k[k['name'].str.match('.*hep.*')]
# 1-methyl-3-hexenyl acetate => CCC=CCC(C)OC(=O)C
l

,MolecularWeight,IsomericSMILES,IUPACName,name
CID,,,,
8159,158.24,CCCCCCCOC(=O)C,heptyl acetate,heptyl acetate
82493,186.29,CC(C)CC(CC(C)C)OC(=O)C,"2,6-dimethylheptan-4-yl acetate","2,6-dimethylheptan-4-yl acetate"
87947,170.25,CC(CCC=C(C)C)OC(=O)C,6-methylhept-5-en-2-yl acetate,6-methylhept-5-en-2-yl acetate
97987,172.26,CCCCCCC(C)OC(=O)C,octan-2-yl acetate,1-methylheptyl acetate
118088,194.27,CC1=CC(C2CC1C2(C)C)OC(=O)C,"(4,6,6-trimethyl-2-bicyclo[3.1.1]hept-3-enyl) ...","4,6,6-trimethylbicyclo[3.1.1]hept-3-en-2-yl ac..."
537615,158.24,CCCCC(CC)OC(=O)C,heptan-3-yl acetate,heptan-3-yl acetate
5352813,156.22,CC/C=C/CC(C)OC(=O)C,[(E)-hept-4-en-2-yl] acetate,"4-hepten-2-ol, (e)-, acetate"
5363206,156.22,CCC/C=C/CCOC(=O)C,[(E)-hept-3-enyl] acetate,(e)-hept-3-enyl acetate


In [11]:
# l.apply(lambda x: mol_img(x['IsomericSMILES'], x['name']), axis=1)

analysis continues

In [12]:
molecules.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3522 entries, -955348933095 to 162353069
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   MolecularWeight  3522 non-null   float64
 1   IsomericSMILES   3522 non-null   object 
 2   IUPACName        3486 non-null   object 
 3   name             3522 non-null   object 
dtypes: float64(1), object(3)
memory usage: 137.6+ KB


In [13]:
len(molecules.drop_duplicates())

3522

In [14]:
total_molecules = len(molecules['IsomericSMILES'].value_counts())
total_molecules

3522

In [15]:
from rdkit import Chem

def canonical_smiles(smiles):
    return Chem.MolToSmiles(Chem.MolFromSmiles(smiles), isomericSmiles = True)

In [16]:
canonical_smiles('CCC=CCC(C)OC(=O)C')

'CCC=CCC(C)OC(C)=O'

In [17]:
molecules['canonicalSMILES'] = molecules['IsomericSMILES'].progress_apply(lambda x: canonical_smiles(x))
assert len(molecules['canonicalSMILES'].value_counts()) == total_molecules

100%|██████████| 3522/3522 [00:00<00:00, 9229.19it/s]


In [18]:
molecules = molecules.drop(columns=['canonicalSMILES'])

In [19]:
molecules.columns

Index(['MolecularWeight', 'IsomericSMILES', 'IUPACName', 'name'], dtype='object')

In [20]:
molecules['IsomericSMILES'][-862841422647]

'CCCC(S)COCC'

possible tests: behavior
- basic
- count of unique behaviours available, maybe pivot table
- % count of each behaviour (sorted)

In [21]:
behavior.head()

,alcoholic,aldehydic,alliaceous,almond,animal,anisic,apple,apricot,aromatic,balsamic,...,tobacco,tomato,tropical,vanilla,vegetable,violet,warm,waxy,winey,woody
Stimulus,,,,,,,,,,,,,,,,,,,,,
-955348933095,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
-923209957509,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
-874408321546,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
-873963935677,0,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
-862841422647,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
behavior.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3522 entries, -955348933095 to 162353069
Columns: 113 entries, alcoholic to woody
dtypes: int64(113)
memory usage: 3.1 MB


In [23]:
odors = list(behavior.columns)
odors

['alcoholic',
 'aldehydic',
 'alliaceous',
 'almond',
 'animal',
 'anisic',
 'apple',
 'apricot',
 'aromatic',
 'balsamic',
 'banana',
 'beefy',
 'berry',
 'black currant',
 'brandy',
 'bread',
 'brothy',
 'burnt',
 'buttery',
 'cabbage',
 'camphoreous',
 'caramellic',
 'catty',
 'chamomile',
 'cheesy',
 'cherry',
 'chicken',
 'chocolate',
 'cinnamon',
 'citrus',
 'cocoa',
 'coconut',
 'coffee',
 'cognac',
 'coumarinic',
 'creamy',
 'cucumber',
 'dairy',
 'dry',
 'earthy',
 'ethereal',
 'fatty',
 'fermented',
 'fishy',
 'floral',
 'fresh',
 'fruity',
 'garlic',
 'gasoline',
 'grape',
 'grapefruit',
 'grassy',
 'green',
 'hay',
 'hazelnut',
 'herbal',
 'honey',
 'horseradish',
 'jasmine',
 'ketonic',
 'leafy',
 'leathery',
 'lemon',
 'malty',
 'meaty',
 'medicinal',
 'melon',
 'metallic',
 'milky',
 'mint',
 'mushroom',
 'musk',
 'musty',
 'nutty',
 'odorless',
 'oily',
 'onion',
 'orange',
 'orris',
 'peach',
 'pear',
 'phenolic',
 'pine',
 'pineapple',
 'plum',
 'popcorn',
 'potato',


In [24]:
len(odors)

113

In [25]:
pd.DataFrame(behavior.sum(axis=1).sort_values()).groupby(by=0).size()
# number of desc | number of molecules

0
1    301
2    527
3    860
4    834
5    568
6    300
7    103
8     24
9      5
dtype: int64

In [26]:
# molecules with no descriptors
pd.DataFrame(behavior.sum(axis=1).sort_values(), columns=['count']).query('count==0')

,count
Stimulus,


In [27]:
# molecules with max (9) descriptors
max_desc_molecules = list(dict(pd.DataFrame(behavior.sum(axis=1).sort_values(), columns=['count']).query('count==9')['count']).keys())
max_desc_molecules

[43263, 95594, 101363, 5463910, 605629]

In [28]:

behavior_sparse['Labels'][max_desc_molecules]

Stimulus
43263      ['ethereal', 'green', 'fruity', 'banana', 'sha...
95594      ['citrus', 'green', 'fruity', 'melon', 'almond...
101363     ['citrus', 'ethereal', 'oily', 'fruity', 'fatt...
5463910    ['rose', 'citrus', 'green', 'fruity', 'melon',...
605629     ['creamy', 'green', 'fruity', 'milky', 'vanill...
Name: Labels, dtype: object

In [29]:
behavior_wise = pd.melt(behavior)

In [30]:
behavior_wise.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397986 entries, 0 to 397985
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   variable  397986 non-null  object
 1   value     397986 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 6.1+ MB


In [31]:
behavior_wise['value'].value_counts()

value
0    385117
1     12869
Name: count, dtype: int64

In [32]:
# behavior.sum().sort_values(ascending=False) # same result
behavior_wise.groupby(by='variable').agg(sum).sort_values(by="value", ascending=False)

,value
variable,
fruity,1391
green,907
sweet,825
floral,553
fatty,407
...,...
horseradish,20
leathery,20
radish,20


In [33]:
behavior_sparse.head()


,Raw Labels,Labels
Stimulus,,
-955348933095,"Herbal-green, waxy, oily, fruity","['green', 'oily', 'fruity', 'waxy', 'herbal']"
-923209957509,"Herbaceous, woody, slight spicy fruity odor","['woody', 'spicy', 'fruity', 'herbal']"
-874408321546,"delta-decalactone precursor; peach, apricot, b...","['peach', 'apricot', 'buttery']"
-873963935677,"Green, fruity, apple-like","['green', 'fruity', 'apple', 'tropical']"
-862841422647,Catty urine; cassis in dilution,['catty']


In [34]:
behavior_sparse.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3522 entries, -955348933095 to 162353069
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Raw Labels  3522 non-null   object
 1   Labels      3522 non-null   object
dtypes: object(2)
memory usage: 211.6+ KB


In [35]:
behavior_sparse['descriptors'] = behavior_sparse['Labels'].apply(lambda x: ';'.join(eval(x)))

In [36]:
behavior_sparse['Raw Labels'][41570]

'Enhances creamy, cocoa aroma; also enhences meaty flavors'

In [37]:
behavior_sparse.head()

,Raw Labels,Labels,descriptors
Stimulus,,,
-955348933095,"Herbal-green, waxy, oily, fruity","['green', 'oily', 'fruity', 'waxy', 'herbal']",green;oily;fruity;waxy;herbal
-923209957509,"Herbaceous, woody, slight spicy fruity odor","['woody', 'spicy', 'fruity', 'herbal']",woody;spicy;fruity;herbal
-874408321546,"delta-decalactone precursor; peach, apricot, b...","['peach', 'apricot', 'buttery']",peach;apricot;buttery
-873963935677,"Green, fruity, apple-like","['green', 'fruity', 'apple', 'tropical']",green;fruity;apple;tropical
-862841422647,Catty urine; cassis in dilution,['catty'],catty


spell correct

In [38]:
# find words
def find(string):
    if len(string.split(';')) > 2:
        print(string)
behavior_sparse['Raw Labels'].apply(lambda x: find(x))

Sulfurous; meaty on dilution; HVP, liver
Nearly odorless; slight tingling taste; sweetness modiffier
Pungent, breathtaking; in dilution a nutty alcoholic note; enhances citrus
Odorless; sour taste; used in reaction flavors for meat
Almost odorless; bland waxy taste; adds mouthfeel
Burnt-rubber; meaty, poultry-like in dilution; modifier for fruit flavors
Odorless; faint, bland taste; adds mouthfeel to low fat products
Very weak ethereal-fruity odor;a mild sweet taste; normal use as solvent
Essentially odorless; slightly bitter taste; used in reaction flavors
Sweet, herbaceous-floral; orchid like odor; sweet, spice, fruity taste
Strong, sweet floral-honey odor; sweet honey-like taste; slightly spicy
Fatty-rancid; in dilution sweet, nutty, green, fatty-cognac like; citrus
Faint, waxy, sweet odor; nearly tasteless; oily-creamy mouthfeel
Almost odorless; oily-waxy; adds mouthfeel
Animal-musty odor; burnt-oily, floral-hebaceous on dilution; green rooty
Fatty-waxy; fatty mouthfeel; see commen

Stimulus
-955348933095    None
-923209957509    None
-874408321546    None
-873963935677    None
-862841422647    None
                 ... 
 154241350       None
 155323379       None
 158699780       None
 161504856       None
 162353069       None
Name: Raw Labels, Length: 3522, dtype: object

In [124]:
root_req_desc = required_desc.copy()
for i in range(len(root_req_desc)):
    root_req_desc[i] = root_req_desc[i][:-(int(0.4*len(root_req_desc[i])))]

In [125]:
root_req_desc

['alcoho',
 'aldehy',
 'alliac',
 'almo',
 'amb',
 'anim',
 'anis',
 'app',
 'apric',
 'aroma',
 'balsa',
 'bana',
 'bee',
 'berga',
 'ber',
 'bitt',
 'black cu',
 'bran',
 'bur',
 'butte',
 'cabba',
 'camphor',
 'carame',
 'ced',
 'cele',
 'chamom',
 'chee',
 'cher',
 'chocol',
 'cinna',
 'citr',
 'cle',
 'clo',
 'coc',
 'cocon',
 'coff',
 'cogn',
 'cook',
 'cooli',
 'cort',
 'coumar',
 'crea',
 'cucum',
 'dai',
 'dr',
 'eart',
 'ether',
 'fat',
 'fermen',
 'fis',
 'flor',
 'fre',
 'fruit ',
 'frui',
 'garl',
 'gas',
 'geran',
 'gra',
 'grapef',
 'gras',
 'gre',
 'hawth',
 'ha',
 'hazel',
 'herb',
 'hon',
 'hyaci',
 'jasm',
 'jui',
 'keton',
 'lacto',
 'laven',
 'lea',
 'leath',
 'lem',
 'lil',
 'mal',
 'mea',
 'medici',
 'mel',
 'metal',
 'mil',
 'min',
 'mugu',
 'mushr',
 'mus',
 'mus',
 'natur',
 'nut',
 'odorl',
 'oil',
 'oni',
 'oran',
 'orangefl',
 'orr',
 'ozo',
 'pea',
 'pea',
 'pheno',
 'pin',
 'pineap',
 'plu',
 'popco',
 'pota',
 'powde',
 'punge',
 'radi',
 'raspbe',
 'rip

In [126]:
import enchant

# Create an instance of the English dictionary
dictionary = enchant.Dict("en_US")

# Check if a word is spelled correctly
def spellcheck(word):
    if dictionary.check(word):
        # print(f"{word} is spelled correctly.")
        pass
    else:
        if word in required_desc: # ignore spell error if word in required desc
            return
        for root in root_req_desc:
            if root in word:    # only find spell error for words that have the certain root words
                suggestions = dictionary.suggest(word)
                if suggestions:
                    print(f"{word} is misspelled. Suggestions: {', '.join(suggestions)}")
                else:
                    print(f"{word} is misspelled, but no suggestions available.")
                break

In [127]:
import re
words_set = set()
def get_spell_errors(string):
    string = string.lower()
    string = string.split(';')[0] #choose odor and not flavor
    list_desc=re.sub(r'[!"#$%&\'()*\-+,./:;<=>?@\[\]^_`{|}~]'," ",string).split(" ")

    # # handle black currant
    # if "black" in list_desc:
    #     idx = list_desc.index('black')
    #     if idx<len(list_desc)-1:
    #         if list_desc[idx+1] == 'currant':
    #             list_desc.pop(idx) #removes black
    #             list_desc.pop(idx) # removes currant
    #             list_desc.append("black currant")
    
    for word in list_desc:
        if len(word) == 0:
            continue
        elif word == ' ':
            continue
        else:
            words_set.update([word])
    # odor_set = set()
    # for req_odor in required_desc:
    #     if req_odor in list_desc:
    #         odor_set

In [128]:
behavior_sparse['Raw Labels'].apply(lambda x: get_spell_errors(x))

Stimulus
-955348933095    None
-923209957509    None
-874408321546    None
-873963935677    None
-862841422647    None
                 ... 
 154241350       None
 155323379       None
 158699780       None
 161504856       None
 162353069       None
Name: Raw Labels, Length: 3522, dtype: object

In [129]:
list_of_words = list(words_set)
list_of_words.sort()
for word in list_of_words:
    spellcheck(word)

acetaldehyde is misspelled. Suggestions: asphalted, statehood, stilted, castellated, interluded, adulated, assaulted, astounded, satellited, escalated, osculated, interlarded, auscultated, stolidity, oscillated, postulated, adulterate, isolated
acetataldehyde is misspelled, but no suggestions available.
acetophenone is misspelled. Suggestions: acetone, Stephenson, acetylene, antiphon, antivenin, acetone's, Stephen, antiphons, Stephanie, Stephens, adenine, atoning, Stephen's, acetaminophen, antiphon's, acetylene's
agrestic is misspelled. Suggestions: acrostic, agnostic, acrostics, aphoristic, acoustic, egoistic, juristic, eagerest, orgiastic, acrostic's
alliaceious is misspelled. Suggestions: fallacious, alliances, malicious, salacious, alliance's, Alice's, aliases, siliceous, Delicious, audacious, delicious, luscious, alliance, fallacies, Alsace, Eliseo's, Alyce's, ulcerous, Allison's
alliacious is misspelled. Suggestions: fallacious, malicious, salacious, Delicious, audacious, delicio

In [45]:
# manual spellcheck and filter
spellcorrect = {}
spellcorrect['acetaldehyde'] = ['acetataldehyde']
spellcorrect['alliaceous'] = ['alliaceious', 'alliacious', 'alliacous']
spellcorrect['amber'] = ['ambery']
spellcorrect['animal'] = ['animalic', 'animalistic', 'animallic']
spellcorrect['apple'] = ['applle']
spellcorrect['balsamic'] = ['balsum']
spellcorrect['black currant'] = ['blackurrant']
spellcorrect['buttery'] = ['butterry']
spellcorrect['camphoreous'] = ['camhorous', 'camphaceous', 'camphoraceous', 'camphoraceousl', 'camphorous']
spellcorrect['caramellic'] = ['carameellic', 'caramelic', 'caramellized']
spellcorrect['cedar'] = ['cedary']
spellcorrect['chamomile'] = ['camomille', 'chamomille']
spellcorrect['cheesy'] = ['cheesey']
spellcorrect['cocoa'] = ['tcocoa']
spellcorrect['cinnamon'] = ['cinnamate', 'cinnamic', 'cinnamyl']
spellcorrect['citrus'] = ['cistus', 'citral', 'citronellal', 'citronellol', 'citrusy']
spellcorrect['coumarinic'] = ['coumarin']
spellcorrect['creamy'] = ['ceamy']
spellcorrect['earthy'] = ['eartthy', 'eathy']
spellcorrect['ethereal'] = ['etheral', 'etthereal']
spellcorrect['fermented'] = ['fermenty', 'ferrmented']
spellcorrect['fruity'] = ['frui', 'fruiity', 'fruiti', 'fruitti', 'fruiy', 'frutti', 'fruty', 'fuity', 'ruity']
spellcorrect['floral'] = ['floralodor', 'flouve', 'foral']
spellcorrect['green'] = ['freen', 'grreen', 'geen']
spellcorrect['grapefruit'] = ['grapefuit']
spellcorrect['hawthorn'] = ['hawthorne']
spellcorrect['hay'] = ['haylike']
spellcorrect['herbal'] = ['hebal', 'herby', 'hebaceous']
spellcorrect['honey'] = ['honeylike']
spellcorrect['jasmin'] = ['jasmone']
spellcorrect['lemon'] = ['iime', 'limonene']
spellcorrect['lactonic'] = ['decalactone', 'dodecalactone', 'iactonic', 'lactone', 'octalactone']
spellcorrect['malty'] = ['maltol']
spellcorrect['milky'] = ['mildl', 'milkfat']
spellcorrect['mushroom'] = ['mushoom']
spellcorrect['musk'] = ['ambrette']
spellcorrect['onion'] = ['oinion']
spellcorrect['odorless'] = ['odourless']
spellcorrect['orange blossom'] = ['orangeblossom']
spellcorrect['orange'] = ['ornge'] 
spellcorrect['ozone'] = ['ozonic']
spellcorrect['passion fruit'] = ['passionfruit']
spellcorrect['pineapple'] = ['pienapple', 'pieneapple']
spellcorrect['popcorn'] = ['poocorn']
spellcorrect['pungent'] = ['pumgent']
spellcorrect['pine'] = ['pune', 'pne']
spellcorrect['roasted'] = ['roasty']
spellcorrect['rose'] = ['rosemay', 'rosey']
spellcorrect['savory'] = ['savoury']
spellcorrect['solvent'] = ['solventy']
spellcorrect['strawberry'] = ['strawberr']
spellcorrect['sulfurous'] = ['sufurous', 'sulfuraceos', 'sulfuraceous', 'sulfureous', 'sulfury', 'sulphur', 'sulphurol', 'sulphurous', 'sulphury']
spellcorrect['spicy'] = ['sicy']
spellcorrect['tobacco'] = ['tabac']
spellcorrect['terpenic'] = ['terpene', 'terpenes', 'terpentine', 'terpeny', 'terpineol', 'terpy']
spellcorrect['tropical'] = ['ltropical', 'tropicall']
spellcorrect['vanilla'] = ['vanillic', 'vanillin', 'vanlla']
spellcorrect['vegetable'] = ['vegetal']
spellcorrect['vetiver'] = ['vertivert', 'vetivert', 'vetiveryl']

In [46]:
spellcorrect

{'acetaldehyde': ['acetataldehyde'],
 'alliaceous': ['alliaceious', 'alliacious', 'alliacous'],
 'amber': ['ambery'],
 'animal': ['animalic', 'animalistic', 'animallic'],
 'apple': ['applle'],
 'balsamic': ['balsum'],
 'black currant': ['blackurrant'],
 'buttery': ['butterry'],
 'camphoreous': ['camhorous',
  'camphaceous',
  'camphoraceous',
  'camphoraceousl',
  'camphorous'],
 'caramellic': ['carameellic', 'caramelic', 'caramellized'],
 'cedar': ['cedary'],
 'chamomile': ['camomille', 'chamomille'],
 'cheesy': ['cheesey'],
 'cocoa': ['tcocoa'],
 'cinnamon': ['cinnamate', 'cinnamic', 'cinnamyl'],
 'citrus': ['cistus', 'citral', 'citronellal', 'citronellol', 'citrusy'],
 'coumarinic': ['coumarin'],
 'creamy': ['ceamy'],
 'earthy': ['eartthy', 'eathy'],
 'ethereal': ['etheral', 'etthereal'],
 'fermented': ['fermenty', 'ferrmented'],
 'fruity': ['frui',
  'fruiity',
  'fruiti',
  'fruitti',
  'fruiy',
  'frutti',
  'fruty',
  'fuity',
  'ruity'],
 'floral': ['floralodor', 'flouve', 'for

In [47]:
# correct spell errors trial 1
from collections import defaultdict
reverse_spellcorrect = defaultdict(list)
for key, values in spellcorrect.items():
    for value in values:
        reverse_spellcorrect[value].append(key)

In [48]:
'ornge' in reverse_spellcorrect.keys()

True

In [49]:
if 'sulfuraceous' in reverse_spellcorrect.keys():
    print(reverse_spellcorrect['sulfuraceous'])

['sulfurous']


In [50]:
import re
def correct_spell_errors_v1(string):
    string = string.lower()
    string = string.split(';')[0] #choose odor and not flavor
    list_desc=re.sub(r'[!"#$%&\'()*\-+,./:;<=>?@\[\]^_`{|}~]'," ",string).split(" ")

    # handle black currant
    if "black" in list_desc:
        idx = list_desc.index('black')
        if idx<len(list_desc)-1:
            if list_desc[idx+1] == 'currant':
                list_desc.pop(idx) #removes black
                list_desc.pop(idx) # removes currant
                list_desc.append("black currant")
    
    # handle orange flower
    if "orange" in list_desc:
        idx = list_desc.index('orange')
        if idx<len(list_desc)-1:
            if list_desc[idx+1] == 'flower':
                list_desc.pop(idx) #removes orange
                list_desc.pop(idx) # removes flower
                list_desc.append("orangeflower")
    
    # handle fruit skin
    for idx in range(len(list_desc)):
        if list_desc[idx] == 'skin' or list_desc[idx] == 'peel':
            if idx != 0:
                if list_desc[idx-1] in ['citrus', 'orange', 'apple', 'banana', 'lemon', 'pear', 'fruit', 'peach']:
                    list_desc.append("fruit skin")


    words_set = set()
    for word in list_desc:
        flag = 0
        if len(word) == 0:
            continue
        elif word == ' ':
            continue
        else:
            for mispelled, corrected in reverse_spellcorrect.items():
                if word == mispelled:
                    words_set.update(corrected)
                    flag = 1
                    break
            if flag == 0:
                words_set.update([word])
    new_string = " ".join(words_set)
    return new_string


In [51]:
correct_spell_errors_v1('sulfuraceous, ltropical-over-ripe fruity, strawberry, cream & cheese notes')

'cream tropical fruity ripe sulfurous notes strawberry over cheese'

In [52]:
correct_spell_errors_v1('sulfury, alliaceous, roasted, savoury, truffle, pne')

'roasted savory alliaceous sulfurous truffle pine'

In [53]:
behavior_sparse['cleaned raw labels'] = behavior_sparse['Raw Labels'].apply(lambda x: correct_spell_errors_v1(x))

In [54]:
behavior_sparse.head()

,Raw Labels,Labels,descriptors,cleaned raw labels
Stimulus,,,,
-955348933095,"Herbal-green, waxy, oily, fruity","['green', 'oily', 'fruity', 'waxy', 'herbal']",green;oily;fruity;waxy;herbal,oily fruity waxy herbal green
-923209957509,"Herbaceous, woody, slight spicy fruity odor","['woody', 'spicy', 'fruity', 'herbal']",woody;spicy;fruity;herbal,fruity woody spicy slight odor herbaceous
-874408321546,"delta-decalactone precursor; peach, apricot, b...","['peach', 'apricot', 'buttery']",peach;apricot;buttery,precursor lactonic delta
-873963935677,"Green, fruity, apple-like","['green', 'fruity', 'apple', 'tropical']",green;fruity;apple;tropical,like green fruity apple
-862841422647,Catty urine; cassis in dilution,['catty'],catty,urine catty


spell correction round 2

In [55]:
suff_root_req_desc = required_desc.copy()
for i in range(len(suff_root_req_desc)):
    suff_root_req_desc[i] = suff_root_req_desc[i][(int(0.4*len(suff_root_req_desc[i]))):]

# tried with different values like: 0.4, 0.6, 0.3

In [56]:
suff_root_req_desc

['oholic',
 'ehydic',
 'aceous',
 'mond',
 'ber',
 'imal',
 'isic',
 'ple',
 'ricot',
 'matic',
 'samic',
 'nana',
 'efy',
 'gamot',
 'rry',
 'tter',
 ' currant',
 'andy',
 'rnt',
 'ttery',
 'bbage',
 'horeous',
 'mellic',
 'dar',
 'lery',
 'momile',
 'eesy',
 'erry',
 'colate',
 'namon',
 'trus',
 'ean',
 'ove',
 'coa',
 'conut',
 'ffee',
 'gnac',
 'oked',
 'oling',
 'rtex',
 'arinic',
 'eamy',
 'umber',
 'iry',
 'ry',
 'rthy',
 'ereal',
 'tty',
 'mented',
 'shy',
 'oral',
 'esh',
 't skin',
 'uity',
 'rlic',
 'ssy',
 'anium',
 'ape',
 'efruit',
 'assy',
 'een',
 'thorn',
 'ay',
 'elnut',
 'rbal',
 'ney',
 'cinth',
 'smin',
 'icy',
 'tonic',
 'tonic',
 'ender',
 'afy',
 'thery',
 'mon',
 'ily',
 'lty',
 'aty',
 'icinal',
 'lon',
 'allic',
 'lky',
 'int',
 'guet',
 'hroom',
 'usk',
 'sty',
 'tural',
 'tty',
 'rless',
 'ily',
 'ion',
 'ange',
 'geflower',
 'ris',
 'one',
 'ach',
 'ear',
 'nolic',
 'ine',
 'eapple',
 'lum',
 'pcorn',
 'tato',
 'wdery',
 'ngent',
 'dish',
 'pberry',
 'ipe

In [57]:
def spellcheck(word):
    if dictionary.check(word):
        # print(f"{word} is spelled correctly.")
        pass
    else:
        if word in required_desc: # ignore spell error if word in required desc
            return
        for root in suff_root_req_desc:
            if root in word:    # only find spell error for words that have the certain root words
                suggestions = dictionary.suggest(word)
                if suggestions:
                    print(f"{word} is misspelled. Suggestions: {', '.join(suggestions)}")
                else:
                    print(f"{word} is misspelled, but no suggestions available.")
                break

In [58]:
import re
words_set_v2 = set()
def get_spell_errors_v2(string):
    string = string.lower()
    list_desc=string.split(" ")

    # handle black currant
    if "black" in list_desc:
        idx = list_desc.index('black')
        if idx<len(list_desc)-1:
            if list_desc[idx+1] == 'currant':
                list_desc.pop(idx) #removes black
                list_desc.pop(idx) # removes currant
                list_desc.append("black currant")

    # handle orange flower
    if "orange" in list_desc:
        idx = list_desc.index('orange')
        if idx<len(list_desc)-1:
            if list_desc[idx+1] == 'flower':
                list_desc.pop(idx) #removes orange
                list_desc.pop(idx) # removes flower
                list_desc.append("orangeflower")
    
    # handle fruit skin
    for idx in range(len(list_desc)):
        if list_desc[idx] == 'skin' or list_desc[idx] == 'peel':
            if idx != 0:
                if list_desc[idx-1] in ['citrus', 'orange', 'apple', 'banana', 'lemon', 'pear', 'fruit', 'peach']:
                    list_desc.append("fruit skin")
    
    for word in list_desc:
        if len(word) == 0:
            continue
        elif word == ' ':
            continue
        else:
            words_set_v2.update([word])
    # odor_set = set()
    # for req_odor in required_desc:
    #     if req_odor in list_desc:
    #         odor_set

In [59]:
behavior_sparse['cleaned raw labels'].apply(lambda x: get_spell_errors_v2(x))

Stimulus
-955348933095    None
-923209957509    None
-874408321546    None
-873963935677    None
-862841422647    None
                 ... 
 154241350       None
 155323379       None
 158699780       None
 161504856       None
 162353069       None
Name: cleaned raw labels, Length: 3522, dtype: object

In [60]:
list_of_words = list(words_set_v2)
list_of_words.sort()
for word in list_of_words:
    spellcheck(word)

acetophenone is misspelled. Suggestions: acetone, Stephenson, acetylene, antiphon, antivenin, acetone's, Stephen, antiphons, Stephanie, Stephens, adenine, atoning, Stephen's, acetaminophen, antiphon's, acetylene's
ammoniacal is misspelled. Suggestions: ammonia cal, ammonia-cal, demoniacal, maniacal, demoniacally, maniacally, ironical, harmonically, demonically, ecumenical
beany is misspelled. Suggestions: Bean, bean, Benny, beans, Meany, beady, meany, Ben, ban, beanie, Bern, bane, bani, been, bony, Barney, barney, barn, being, bonny, bunny, be any, be-any, brainy, brawny, Beau, Behan, Bran, bandy, beau, began, bendy, bent, bran, belay, Bethany, Ebony, bay, bey, ebony, any, Bean's, bean's, beaned, botany, bin, bun, Benz, band, bang, bank, bans, bear, bend, Ben's, ban's, Benny's
breadcrust is misspelled. Suggestions: bread crust, bread-crust, broadcast, breadcrumb, breadfruit, breadfruits, readjust, breadfruit's
bready is misspelled. Suggestions: Brady, bread, beady, ready, breads, Brad,

In [61]:
# # manual spellcheck and filter
# spellcorrect_v2 = {}
# spellcorrect_v2['creamy'] = ['ceamy']
# spellcorrect_v2['green'] = ['geen']
# spellcorrect_v2['herbal'] = ['hebaceous']
# spellcorrect_v2['spicy'] = ['sicy']
# spellcorrect_v2['pine'] = ['pne']

# added to previous spell correction

In [62]:
behavior_sparse.head()

,Raw Labels,Labels,descriptors,cleaned raw labels
Stimulus,,,,
-955348933095,"Herbal-green, waxy, oily, fruity","['green', 'oily', 'fruity', 'waxy', 'herbal']",green;oily;fruity;waxy;herbal,oily fruity waxy herbal green
-923209957509,"Herbaceous, woody, slight spicy fruity odor","['woody', 'spicy', 'fruity', 'herbal']",woody;spicy;fruity;herbal,fruity woody spicy slight odor herbaceous
-874408321546,"delta-decalactone precursor; peach, apricot, b...","['peach', 'apricot', 'buttery']",peach;apricot;buttery,precursor lactonic delta
-873963935677,"Green, fruity, apple-like","['green', 'fruity', 'apple', 'tropical']",green;fruity;apple;tropical,like green fruity apple
-862841422647,Catty urine; cassis in dilution,['catty'],catty,urine catty


get labels

In [63]:
from collections import defaultdict
roots_dict = defaultdict(set)
def connect_roots(cleaned_raw_labels):
    raw_list = cleaned_raw_labels.split(' ')
    for desc in required_desc:
        root_set = set()
        for word in raw_list:
            if word == desc:
                root_set.update([word])
            elif re.match(f'{desc[:-(int(0.6*len(desc)))]}.*', word):
                root_set.update([word])
        roots_dict[desc].update(list(root_set))
    

In [64]:
behavior_sparse['cleaned raw labels'].apply(lambda x: connect_roots(x))

Stimulus
-955348933095    None
-923209957509    None
-874408321546    None
-873963935677    None
-862841422647    None
                 ... 
 154241350       None
 155323379       None
 158699780       None
 161504856       None
 162353069       None
Name: cleaned raw labels, Length: 3522, dtype: object

In [65]:
roots_dict

defaultdict(set,
            {'alcoholic': {'alcohol', 'alcoholic'},
             'aldehydic': {'aldehydic'},
             'alliaceous': {'alliaceous', 'allium'},
             'almond': {'almond', 'almonds', 'almost'},
             'amber': {'amber',
              'ambergris',
              'amine',
              'ammonia',
              'ammoniacal',
              'amyl'},
             'animal': {'animal', 'anise', 'anisic', 'anisole'},
             'anisic': {'animal', 'anise', 'anisic', 'anisole'},
             'apple': {'apple', 'apricot'},
             'apricot': {'apricot'},
             'aromatic': {'aroma', 'aromas', 'aromatic'},
             'balsamic': {'balsam', 'balsamic'},
             'banana': {'banana'},
             'beefy': {'bean',
              'beans',
              'beany',
              'becomes',
              'beef',
              'beefsteak',
              'beefy',
              'beer',
              'bell',
              'benzaldehyde',
              'benzene

In [66]:
# manually cleaned desc root combination
merger_root_dict = {}
merger_root_dict['alcoholic'] = ['alcohol', 'alcoholic']
merger_root_dict['aldehydic'] = ['aldehydic']
merger_root_dict['alliaceous'] = ['alliaceous', 'allium']
merger_root_dict['almond'] = ['almond', 'almonds']
merger_root_dict['amber'] = ['amber', 'ambergris']
merger_root_dict['animal'] = ['animal']
merger_root_dict['anisic'] = ['anise', 'anisic']
merger_root_dict['apple'] = ['apple']
merger_root_dict['apricot'] = ['apricot']
merger_root_dict['aromatic'] = ['aromatic']
merger_root_dict['balsamic'] = ['balsam', 'balsamic']
merger_root_dict['banana'] = ['banana']
merger_root_dict['beefy'] = ['beef', 'beefsteak', 'beefy']
merger_root_dict['bergamot'] = ['bergamot']
merger_root_dict['berry'] = ['berry']
merger_root_dict['bitter'] = ['bitter', 'bitterness', 'bittersweet']
merger_root_dict['black currant'] = ['black currant']
merger_root_dict['brandy'] = ['brandy']
merger_root_dict['burnt'] = ['burned', 'burning', 'burnt']
merger_root_dict['buttery'] = ['butter', 'buttery']
merger_root_dict['cabbage'] = ['cabbage']
merger_root_dict['camphoreous'] = ['camphor', 'camphoreous']
merger_root_dict['caramellic'] = ['caramel', 'caramelized', 'caramellic']
merger_root_dict['cedar'] = ['cedar']
merger_root_dict['celery'] = ['celery']
merger_root_dict['chamomile'] = ['chamomile']
merger_root_dict['cheesy'] = ['cheese', 'cheesy']
merger_root_dict['cherry'] = ['cherries', 'cherry']
merger_root_dict['chocolate'] = ['chocolate']
merger_root_dict['cinnamon'] = ['cinnamon']
merger_root_dict['citrus'] = ['citronella', 'citrus']
merger_root_dict['clean'] = ['clean']
merger_root_dict['clove'] = ['clove', 'clover']
merger_root_dict['cocoa'] = ['cocoa']
merger_root_dict['coconut'] = ['coconut']
merger_root_dict['coffee'] = ['coffee']
merger_root_dict['cognac'] = ['cognac']
merger_root_dict['cooked'] = ['cooked']
merger_root_dict['cooling'] = ['cool', 'cooling', 'coolness']
merger_root_dict['cortex'] = ['cortex']
merger_root_dict['coumarinic'] = ['coumarinic']
merger_root_dict['creamy'] = ['cream', 'creamy']
merger_root_dict['cucumber'] = ['cucumber']
merger_root_dict['dairy'] = ['dairy']
merger_root_dict['dry'] = ['dried', 'dry']
merger_root_dict['earthy'] = ['earth', 'earthy']
merger_root_dict['ethereal'] = ['ethereal']
merger_root_dict['fatty'] = ['fat', 'fats', 'fatty']
merger_root_dict['fermented'] = ['fermented', 'fermenting']
merger_root_dict['fishy'] = ['fish', 'fishy']
merger_root_dict['floral'] = ['flora', 'floral', 'flower', 'flowery']
merger_root_dict['fresh'] = ['fresh', 'freshly', 'freshness']
merger_root_dict['fruit skin'] = ['fruit skin']
merger_root_dict['fruity'] = ['fruit', 'fruitier', 'fruitiness', 'fruits', 'fruity']
merger_root_dict['garlic'] = ['garlic']
merger_root_dict['gassy'] = ['gassy']
merger_root_dict['geranium'] = ['geranium']
merger_root_dict['grape'] = ['grape', 'grapes']
merger_root_dict['grapefruit'] = ['grapefruit']
merger_root_dict['grassy'] = ['grass', 'grassy']
merger_root_dict['green'] = ['green', 'greenery', 'greenish']
merger_root_dict['hawthorn'] = ['hawthorn']
merger_root_dict['hay'] = ['hay']
merger_root_dict['hazelnut'] = ['hazel', 'hazelnut', 'hazelnuts']
merger_root_dict['herbal'] = ['herb', 'herbaceous', 'herbal']
merger_root_dict['honey'] = ['honey', 'honeydew', 'honeysuckle']
merger_root_dict['hyacinth'] = ['hyacinth']
merger_root_dict['jasmin'] = ['jasmin', 'jasmine']
merger_root_dict['juicy'] = ['juice', 'juiciness', 'juicy']
merger_root_dict['ketonic'] = ['ketonic']
merger_root_dict['lactonic'] = ['lactonic']
merger_root_dict['lavender'] = ['lavender']
merger_root_dict['leafy'] = ['leaf', 'leafy', 'lettuce']
merger_root_dict['leathery'] = ['leather', 'leathery']
merger_root_dict['lemon'] = ['lemon', 'lemony', 'lime']
merger_root_dict['lily'] = ['lilac', 'lily']
merger_root_dict['malty'] = ['malt', 'malty']
merger_root_dict['meaty'] = ['meat', 'meats', 'meaty']
merger_root_dict['medicinal'] = ['medicinal']
merger_root_dict['melon'] = ['melon']
merger_root_dict['metallic'] = ['metallic']
merger_root_dict['milky'] = ['milk', 'milky']
merger_root_dict['mint'] = ['mint', 'minty']
merger_root_dict['muguet'] = ['muguet']
merger_root_dict['mushroom'] = ['mushroom']
merger_root_dict['musk'] = ['musk', 'musky']
merger_root_dict['musty'] = ['musty']
merger_root_dict['natural'] = ['natural']
merger_root_dict['nutty'] = ['nut', 'nuts', 'nutty']
merger_root_dict['odorless'] = ['odorless']
merger_root_dict['oily'] = ['oil', 'oils', 'oily']
merger_root_dict['onion'] = ['onion', 'onions']
merger_root_dict['orange'] = ['orange']
merger_root_dict['orangeflower'] = ['orangeflower']
merger_root_dict['orris'] = ['orris']
merger_root_dict['ozone'] = ['ozone']
merger_root_dict['peach'] = ['peach', 'peachy']
merger_root_dict['pear'] = ['pear']
merger_root_dict['phenolic'] = ['phenol', 'phenolc', 'phenolic']
merger_root_dict['pine'] = ['pine', 'piney']
merger_root_dict['pineapple'] = ['pineapple']
merger_root_dict['plum'] = ['plum']
merger_root_dict['popcorn'] = ['popcorn']
merger_root_dict['potato'] = ['potato']
merger_root_dict['powdery'] = ['powdery']
merger_root_dict['pungent'] = ['pungency', 'pungent']
merger_root_dict['radish'] = ['radish', 'horseradish']
merger_root_dict['raspberry'] = ['raspberry']
merger_root_dict['ripe'] = ['ripe', 'ripened']
merger_root_dict['roasted'] = ['roast', 'roasted']
merger_root_dict['rose'] = ['rose', 'rosemary', 'rosy']
merger_root_dict['rummy'] = ['rum', 'rummy']
merger_root_dict['sandalwood'] = ['sandalwood']
merger_root_dict['savory'] = ['savory']
merger_root_dict['sharp'] = ['sharp']
merger_root_dict['smoky'] = ['smoke', 'smoked', 'smokey', 'smoky']
merger_root_dict['soapy'] = ['soapy']
merger_root_dict['solvent'] = ['solvent']
merger_root_dict['sour'] = ['sour']
merger_root_dict['spicy'] = ['spice', 'spicy']
merger_root_dict['strawberry'] = ['strawberry']
merger_root_dict['sulfurous'] = ['sulfide', 'sulfur', 'sulfuric', 'sulfurous']
merger_root_dict['sweaty'] = ['sweat','sweaty']
merger_root_dict['sweet'] = ['sweet', 'sweeter', 'sweetish', 'sweetness']
merger_root_dict['tea'] = ['tea']
merger_root_dict['terpenic'] = ['terpenic']
merger_root_dict['tobacco'] = ['tobacco', 'tobaccos']
merger_root_dict['tomato'] = ['tomato']
merger_root_dict['tropical'] = ['tropical']
merger_root_dict['vanilla'] = ['vanilla']
merger_root_dict['vegetable'] = ['vegetable', 'vegetables', 'vegetative']
merger_root_dict['vetiver'] = ['vetiver']
merger_root_dict['violet'] = ['violet']
merger_root_dict['warm'] = ['warm', 'warming']
merger_root_dict['waxy'] = ['waxy']
merger_root_dict['weedy'] = ['weedy']
merger_root_dict['winey'] = ['wine', 'wines', 'winey']
merger_root_dict['woody'] = ['wood', 'woody']


In [67]:
len(merger_root_dict)

138

In [68]:
from collections import defaultdict
reverse_merger_root_dict = defaultdict(list)
for key, values in merger_root_dict.items():
    for value in values:
        reverse_merger_root_dict[value].append(key)

In [69]:
'juiciness' in reverse_merger_root_dict.keys()

True

In [70]:
def clean_desc(cleaned_raw_labels):
    desc_set = set()
    raw_list = cleaned_raw_labels.split(' ')

    # handle black currant
    if "black" in raw_list:
        idx = raw_list.index('black')
        if idx<len(raw_list)-1:
            if raw_list[idx+1] == 'currant':
                raw_list.pop(idx) #removes black
                raw_list.pop(idx) # removes currant
                raw_list.append("black currant")
    
    # handle orange flower
    if "orange" in raw_list:
        idx = raw_list.index('orange')
        if idx<len(raw_list)-1:
            if raw_list[idx+1] == 'flower':
                raw_list.pop(idx) #removes orange
                raw_list.pop(idx) # removes flower
                raw_list.append("orangeflower")
    
    # handle fruit skin
    for idx in range(len(raw_list)):
        if raw_list[idx] == 'skin' or raw_list[idx] == 'peel':
            if idx != 0:
                if raw_list[idx-1] in ['citrus', 'orange', 'apple', 'banana', 'lemon', 'pear', 'fruit', 'peach']:
                    raw_list.append("fruit skin")

    for word in raw_list:
        for root, desc in reverse_merger_root_dict.items():
            if word == root:
                desc_set.update(desc)
                break
    if len(desc_set) == 0:
        return None
    return ';'.join(desc_set)

In [71]:
clean_desc('fruity odorless herbaceous spicy odor slight woody')

'odorless;fruity;herbal;woody;spicy'

In [72]:
behavior_sparse['new_descriptors'] = behavior_sparse['cleaned raw labels'].apply(lambda x: clean_desc(x))

In [73]:
behavior_sparse.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3522 entries, -955348933095 to 162353069
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Raw Labels          3522 non-null   object
 1   Labels              3522 non-null   object
 2   descriptors         3522 non-null   object
 3   cleaned raw labels  3522 non-null   object
 4   new_descriptors     3488 non-null   object
dtypes: object(5)
memory usage: 294.1+ KB


In [74]:
behavior_sparse.head()

,Raw Labels,Labels,descriptors,cleaned raw labels,new_descriptors
Stimulus,,,,,
-955348933095,"Herbal-green, waxy, oily, fruity","['green', 'oily', 'fruity', 'waxy', 'herbal']",green;oily;fruity;waxy;herbal,oily fruity waxy herbal green,oily;fruity;waxy;herbal;green
-923209957509,"Herbaceous, woody, slight spicy fruity odor","['woody', 'spicy', 'fruity', 'herbal']",woody;spicy;fruity;herbal,fruity woody spicy slight odor herbaceous,herbal;spicy;fruity;woody
-874408321546,"delta-decalactone precursor; peach, apricot, b...","['peach', 'apricot', 'buttery']",peach;apricot;buttery,precursor lactonic delta,lactonic
-873963935677,"Green, fruity, apple-like","['green', 'fruity', 'apple', 'tropical']",green;fruity;apple;tropical,like green fruity apple,green;fruity;apple
-862841422647,Catty urine; cassis in dilution,['catty'],catty,urine catty,None


In [75]:
behavior_sparse[behavior_sparse['new_descriptors'].isna()]

,Raw Labels,Labels,descriptors,cleaned raw labels,new_descriptors
Stimulus,,,,,
-862841422647,Catty urine; cassis in dilution,['catty'],catty,urine catty,None
-848964121442,See comments; components of Milk lactone,['milky'],milky,comments see,None
-86422482515,"Unstable; citrus, fatty, tropical","['fatty', 'citrus', 'tropical']",fatty;citrus;tropical,unstable,None
356,Hydrocarbon odor (gasoline-like),['gasoline'],gasoline,hydrocarbon like odor gasoline,None
4837,"Mild salty taste; Somewhat fishy, ammonia odor",['fishy'],fishy,taste mild salty,None
6736,"Strong, offensive fecal odor; diluted - floral...","['fruity', 'animal', 'floral']",fruity;animal;floral,strong offensive odor fecal,None
7363,"Powerful; burnt, coffee-like, somewhat caramel...","['alliaceous', 'burnt', 'caramellic', 'coffee']",alliaceous;burnt;caramellic;coffee,powerful,None
7438,Caraway-like; sweet spearmint-like odor,['sweet'],sweet,caraway like,None
7894,"Ammoniacal odor; fishy taste, on dilution a bl...",['cheesy'],cheesy,ammoniacal odor,None


In [76]:
def update_desc(descriptors, new_descriptors):
    desc_set = set()
    if new_descriptors is None:
        return descriptors
    else:
        concat_desc = descriptors+';'+new_descriptors
        desc_set.update(concat_desc.split(';'))
    return ';'.join(desc_set)

In [77]:
update_desc('alliaceous;burnt;caramellic;coffee', None)

'alliaceous;burnt;caramellic;coffee'

In [78]:
update_desc('green;fruity;apple;tropical', 'green;fruity;apple')

'tropical;green;fruity;apple'

In [79]:
behavior_sparse['new_descriptors'] = behavior_sparse.apply(lambda x: update_desc(x['descriptors'], x['new_descriptors']), axis=1)

In [80]:
behavior_sparse

,Raw Labels,Labels,descriptors,cleaned raw labels,new_descriptors
Stimulus,,,,,
-955348933095,"Herbal-green, waxy, oily, fruity","['green', 'oily', 'fruity', 'waxy', 'herbal']",green;oily;fruity;waxy;herbal,oily fruity waxy herbal green,oily;fruity;waxy;herbal;green
-923209957509,"Herbaceous, woody, slight spicy fruity odor","['woody', 'spicy', 'fruity', 'herbal']",woody;spicy;fruity;herbal,fruity woody spicy slight odor herbaceous,herbal;spicy;fruity;woody
-874408321546,"delta-decalactone precursor; peach, apricot, b...","['peach', 'apricot', 'buttery']",peach;apricot;buttery,precursor lactonic delta,peach;lactonic;buttery;apricot
-873963935677,"Green, fruity, apple-like","['green', 'fruity', 'apple', 'tropical']",green;fruity;apple;tropical,like green fruity apple,tropical;green;fruity;apple
-862841422647,Catty urine; cassis in dilution,['catty'],catty,urine catty,catty
...,...,...,...,...,...
154241350,"Green, allium (onion, garden cress), savory","['green', 'alliaceous', 'savory', 'onion']",green;alliaceous;savory;onion,allium cress savory onion garden green,savory;green;alliaceous;onion
155323379,"Sulfury, alliaceous, roasted, savory, truffle","['roasted', 'alliaceous', 'savory', 'sulfurous']",roasted;alliaceous;savory;sulfurous,roasted savory alliaceous sulfurous truffle,savory;alliaceous;roasted;sulfurous
158699780,"Sulfuraceous, tropical-over-ripe fruity, straw...","['ripe', 'creamy', 'fruity', 'tropical', 'stra...",ripe;creamy;fruity;tropical;strawberry;sulfuro...,cream tropical fruity ripe sulfurous notes str...,creamy;tropical;cheesy;fruity;ripe;sulfurous;s...


In [81]:
behavior_sparse[behavior_sparse['new_descriptors'].str.match('.*odorless.*')]

,Raw Labels,Labels,descriptors,cleaned raw labels,new_descriptors
Stimulus,,,,,
135,Nearly odorless; slight tingling taste; sweetn...,['odorless'],odorless,odorless nearly,odorless
196,Pleasant sour taste; odorless,['odorless'],odorless,taste pleasant sour,odorless;sour
311,"Odorless with a clean, acid, tart taste",['odorless'],odorless,taste odorless with clean a acid tart,odorless;clean
453,"Odorless with a pleasant, sweet taste",['odorless'],odorless,taste odorless pleasant with a sweet,odorless;sweet
525,Almost odorless with a sharp tart acidic taste,['odorless'],odorless,taste almost odorless acidic with a tart sharp,odorless;sharp
...,...,...,...,...,...
54675810,Odorless with a clean acidulous taste,['odorless'],odorless,taste odorless with acidulous clean a,odorless;clean
54676860,"Odorless with a sharp, pleasant acid taste",['odorless'],odorless,taste odorless pleasant with a acid sharp,odorless;sharp
57353225,Nearly odorless; Delicate floral waxy with ros...,"['rose', 'odorless', 'waxy', 'floral']",rose;odorless;waxy;floral,odorless nearly,odorless;rose;waxy;floral


In [82]:
# handle odorless
def handle_odorless(desc_string):
    desc_list = desc_string.split(';')
    desc_set = set()
    for desc in desc_list:
        if desc == 'odorless':
            continue
        desc_set.update([desc])
    if len(desc_set) == 0:
        desc_set.update(['odorless'])
    return ';'.join(desc_set)

In [83]:
handle_odorless('fruity;spicy;odorless;herbal;woody')

'herbal;spicy;fruity;woody'

In [84]:
handle_odorless('odorless')


'odorless'

In [85]:
behavior_sparse['new_descriptors'] = behavior_sparse['new_descriptors'].apply(lambda x: handle_odorless(x))

In [86]:
behavior_sparse[behavior_sparse['new_descriptors'].str.match('.*odorless.*')]

,Raw Labels,Labels,descriptors,cleaned raw labels,new_descriptors
Stimulus,,,,,
135,Nearly odorless; slight tingling taste; sweetn...,['odorless'],odorless,odorless nearly,odorless
594,Odorless; sour taste; used in reaction flavors...,['odorless'],odorless,odorless,odorless
611,"Odorless, flavor enhancer, umami character",['odorless'],odorless,umami odorless flavor character enhancer,odorless
757,"Nearly odorless, acidic tart taste (see comments)",['odorless'],odorless,taste comments odorless see acidic nearly tart,odorless
875,Odorless with a strong tart acid taste,['odorless'],odorless,taste odorless with a acid strong tart,odorless
985,Almost odorless; bland waxy taste; adds mouthfeel,['odorless'],odorless,odorless almost,odorless
1182,"Almost odorless; sweet, bread-like taste",['odorless'],odorless,odorless almost,odorless
1491,Nearly odorless; sweetness modifier for sucros...,['odorless'],odorless,odorless nearly,odorless
2682,"Odorless; faint, bland taste; adds mouthfeel t...",['odorless'],odorless,odorless,odorless


In [87]:
behavior_sparse

,Raw Labels,Labels,descriptors,cleaned raw labels,new_descriptors
Stimulus,,,,,
-955348933095,"Herbal-green, waxy, oily, fruity","['green', 'oily', 'fruity', 'waxy', 'herbal']",green;oily;fruity;waxy;herbal,oily fruity waxy herbal green,oily;fruity;waxy;herbal;green
-923209957509,"Herbaceous, woody, slight spicy fruity odor","['woody', 'spicy', 'fruity', 'herbal']",woody;spicy;fruity;herbal,fruity woody spicy slight odor herbaceous,spicy;fruity;herbal;woody
-874408321546,"delta-decalactone precursor; peach, apricot, b...","['peach', 'apricot', 'buttery']",peach;apricot;buttery,precursor lactonic delta,peach;lactonic;apricot;buttery
-873963935677,"Green, fruity, apple-like","['green', 'fruity', 'apple', 'tropical']",green;fruity;apple;tropical,like green fruity apple,tropical;green;fruity;apple
-862841422647,Catty urine; cassis in dilution,['catty'],catty,urine catty,catty
...,...,...,...,...,...
154241350,"Green, allium (onion, garden cress), savory","['green', 'alliaceous', 'savory', 'onion']",green;alliaceous;savory;onion,allium cress savory onion garden green,savory;green;alliaceous;onion
155323379,"Sulfury, alliaceous, roasted, savory, truffle","['roasted', 'alliaceous', 'savory', 'sulfurous']",roasted;alliaceous;savory;sulfurous,roasted savory alliaceous sulfurous truffle,savory;alliaceous;roasted;sulfurous
158699780,"Sulfuraceous, tropical-over-ripe fruity, straw...","['ripe', 'creamy', 'fruity', 'tropical', 'stra...",ripe;creamy;fruity;tropical;strawberry;sulfuro...,cream tropical fruity ripe sulfurous notes str...,creamy;tropical;cheesy;fruity;ripe;sulfurous;s...


analysis

In [88]:
mol_behaviors = pd.merge(molecules['IsomericSMILES'], behavior_sparse['new_descriptors'], how='inner', left_index=True, right_index=True)
mol_behaviors = mol_behaviors.rename(columns={'new_descriptors':'descriptors'})
mol_behaviors.head(3)

,IsomericSMILES,descriptors
CID,,
-955348933095,CCCCC=COC(=O)CCCCCCCC,oily;fruity;waxy;herbal;green
-923209957509,CC(=O)OCC1C=CC(C(C)C)CC1,spicy;fruity;herbal;woody
-874408321546,CCCCCCCCC(OC(C)=O)C(=O)OC,peach;lactonic;apricot;buttery


In [89]:
mol_behaviors.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3522 entries, -955348933095 to 162353069
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   IsomericSMILES  3522 non-null   object
 1   descriptors     3522 non-null   object
dtypes: object(2)
memory usage: 211.6+ KB


In [90]:
def get_odors(desc, odor_list):
    odors = desc.split(';')
    for i in odors:
        if i not in odor_list:
            odor_list.append(i)

In [91]:
odor_list = []
mol_behaviors['descriptors'].apply(lambda x: get_odors(x, odor_list))
odor_list.sort()
odor_list

['alcoholic',
 'aldehydic',
 'alliaceous',
 'almond',
 'amber',
 'animal',
 'anisic',
 'apple',
 'apricot',
 'aromatic',
 'balsamic',
 'banana',
 'beefy',
 'bergamot',
 'berry',
 'bitter',
 'black currant',
 'brandy',
 'bread',
 'brothy',
 'burnt',
 'buttery',
 'cabbage',
 'camphoreous',
 'caramellic',
 'catty',
 'cedar',
 'celery',
 'chamomile',
 'cheesy',
 'cherry',
 'chicken',
 'chocolate',
 'cinnamon',
 'citrus',
 'clean',
 'clove',
 'cocoa',
 'coconut',
 'coffee',
 'cognac',
 'cooked',
 'cooling',
 'coumarinic',
 'creamy',
 'cucumber',
 'dairy',
 'dry',
 'earthy',
 'ethereal',
 'fatty',
 'fermented',
 'fishy',
 'floral',
 'fresh',
 'fruit skin',
 'fruity',
 'garlic',
 'gasoline',
 'gassy',
 'geranium',
 'grape',
 'grapefruit',
 'grassy',
 'green',
 'hawthorn',
 'hay',
 'hazelnut',
 'herbal',
 'honey',
 'horseradish',
 'hyacinth',
 'jasmin',
 'jasmine',
 'juicy',
 'ketonic',
 'lactonic',
 'lavender',
 'leafy',
 'leathery',
 'lemon',
 'lily',
 'malty',
 'meaty',
 'medicinal',
 'melo

In [92]:
len(odor_list)

145

In [93]:
og_columns = odor_list
# elements of required desc not in og_columns
missing_desc = []
for req in required_desc:
    if req not in og_columns:
        missing_desc.append(req)
missing_desc

['cortex']

In [94]:
odors_df = pd.DataFrame(odor_list, columns=['desc'])
odors_df

,desc
0,alcoholic
1,aldehydic
2,alliaceous
3,almond
4,amber
...,...
140,warm
141,waxy
142,weedy
143,winey


In [95]:
odors_df[odors_df['desc'].str.match('.*flo.*')]

,desc
53,floral
99,orangeflower


In [96]:
odor_dummies = mol_behaviors['descriptors'].str.get_dummies(sep=';')
odor_dummies['fruity'].head()

CID
-955348933095    1
-923209957509    1
-874408321546    0
-873963935677    1
-862841422647    0
Name: fruity, dtype: int64

In [97]:
mol_behaviors_encoded = pd.concat([mol_behaviors, odor_dummies], axis=1)

In [98]:
mol_behaviors_encoded.head(3)

,IsomericSMILES,descriptors,alcoholic,aldehydic,alliaceous,almond,amber,animal,anisic,apple,...,tropical,vanilla,vegetable,vetiver,violet,warm,waxy,weedy,winey,woody
CID,,,,,,,,,,,,,,,,,,,,,
-955348933095,CCCCC=COC(=O)CCCCCCCC,oily;fruity;waxy;herbal;green,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
-923209957509,CC(=O)OCC1C=CC(C(C)C)CC1,spicy;fruity;herbal;woody,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
-874408321546,CCCCCCCCC(OC(C)=O)C(=O)OC,peach;lactonic;apricot;buttery,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [99]:
mol_behaviors_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3522 entries, -955348933095 to 162353069
Columns: 147 entries, IsomericSMILES to woody
dtypes: int64(145), object(2)
memory usage: 4.1+ MB


In [100]:
encoded_behavior = mol_behaviors_encoded.drop(columns=['IsomericSMILES', 'descriptors'])

In [101]:
pd.DataFrame(encoded_behavior.sum(axis=1).sort_values()).groupby(by=0).size()
# number of desc | number of molecules

0
1     232
2     454
3     779
4     895
5     622
6     368
7     126
8      36
9       9
10      1
dtype: int64

In [102]:
# molecules with no descriptors
pd.DataFrame(encoded_behavior.sum(axis=1).sort_values(), columns=['count']).query('count==0')

,count
CID,


In [103]:
dict(encoded_behavior.sum().sort_values(ascending=False))

{'fruity': 1413,
 'green': 913,
 'sweet': 842,
 'floral': 558,
 'fatty': 412,
 'herbal': 408,
 'sulfurous': 253,
 'waxy': 248,
 'apple': 240,
 'fresh': 240,
 'nutty': 230,
 'meaty': 226,
 'spicy': 214,
 'woody': 213,
 'citrus': 212,
 'oily': 209,
 'tropical': 204,
 'roasted': 195,
 'ethereal': 194,
 'winey': 177,
 'earthy': 174,
 'vegetable': 166,
 'balsamic': 166,
 'caramellic': 165,
 'rose': 165,
 'cheesy': 146,
 'pineapple': 142,
 'burnt': 138,
 'onion': 127,
 'mint': 123,
 'pungent': 119,
 'fermented': 110,
 'buttery': 98,
 'creamy': 93,
 'musty': 92,
 'honey': 91,
 'pear': 91,
 'melon': 86,
 'garlic': 85,
 'savory': 84,
 'alcoholic': 84,
 'phenolic': 81,
 'cocoa': 80,
 'cognac': 78,
 'berry': 73,
 'dairy': 73,
 'camphoreous': 70,
 'vanilla': 70,
 'banana': 70,
 'alliaceous': 69,
 'coffee': 63,
 'orange': 60,
 'rum': 59,
 'mushroom': 59,
 'tobacco': 58,
 'fishy': 58,
 'leafy': 56,
 'sharp': 56,
 'rummy': 56,
 'aromatic': 55,
 'almond': 53,
 'sour': 51,
 'peach': 51,
 'grape': 50,
 

In [104]:
og_columns = list(encoded_behavior.columns)
# elements of required desc not in og_columns
missing_desc = []
for req in required_desc:
    if req not in og_columns:
        missing_desc.append(req)
missing_desc

['cortex']

In [105]:
new_required_desc = required_desc.copy()
for i in missing_desc:
    new_required_desc.remove(i)

In [106]:
def get_req_desc(desc_string):
    desc_list = desc_string.split(';')
    desc_set = set()
    for desc in desc_list:
        if desc in required_desc:
            desc_set.update([desc])
    return ';'.join(desc_set)

In [107]:
get_req_desc('sweet;creamy;brown;caramellic;testing')

'caramellic;creamy;sweet'

In [108]:
mol_behaviors_encoded['Updated_Desc_v2'] = mol_behaviors_encoded['descriptors'].apply(lambda x: get_req_desc(x))

In [109]:
mol_behaviors_encoded.head()

,IsomericSMILES,descriptors,alcoholic,aldehydic,alliaceous,almond,amber,animal,anisic,apple,...,vanilla,vegetable,vetiver,violet,warm,waxy,weedy,winey,woody,Updated_Desc_v2
CID,,,,,,,,,,,,,,,,,,,,,
-955348933095,CCCCC=COC(=O)CCCCCCCC,oily;fruity;waxy;herbal;green,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,oily;fruity;waxy;herbal;green
-923209957509,CC(=O)OCC1C=CC(C(C)C)CC1,spicy;fruity;herbal;woody,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,herbal;spicy;fruity;woody
-874408321546,CCCCCCCCC(OC(C)=O)C(=O)OC,peach;lactonic;apricot;buttery,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,peach;lactonic;apricot;buttery
-873963935677,CCCCC=COC(=O)C(C)CCC,tropical;green;fruity;apple,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,tropical;green;fruity;apple
-862841422647,CCCC(S)COCC,catty,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,


In [110]:
required_leffingwell_dataset = mol_behaviors_encoded[['IsomericSMILES', 'Updated_Desc_v2']+new_required_desc]

In [111]:
required_leffingwell_dataset.head()

,IsomericSMILES,Updated_Desc_v2,alcoholic,aldehydic,alliaceous,almond,amber,animal,anisic,apple,...,tropical,vanilla,vegetable,vetiver,violet,warm,waxy,weedy,winey,woody
CID,,,,,,,,,,,,,,,,,,,,,
-955348933095,CCCCC=COC(=O)CCCCCCCC,oily;fruity;waxy;herbal;green,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
-923209957509,CC(=O)OCC1C=CC(C(C)C)CC1,herbal;spicy;fruity;woody,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
-874408321546,CCCCCCCCC(OC(C)=O)C(=O)OC,peach;lactonic;apricot;buttery,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
-873963935677,CCCCC=COC(=O)C(C)CCC,tropical;green;fruity;apple,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
-862841422647,CCCC(S)COCC,,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [112]:
required_leffingwell_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3522 entries, -955348933095 to 162353069
Columns: 139 entries, IsomericSMILES to woody
dtypes: int64(137), object(2)
memory usage: 3.9+ MB


In [113]:
required_encoded = required_leffingwell_dataset.drop(columns=['IsomericSMILES', 'Updated_Desc_v2'])
required_encoded

,alcoholic,aldehydic,alliaceous,almond,amber,animal,anisic,apple,apricot,aromatic,...,tropical,vanilla,vegetable,vetiver,violet,warm,waxy,weedy,winey,woody
CID,,,,,,,,,,,,,,,,,,,,,
-955348933095,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
-923209957509,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
-874408321546,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
-873963935677,0,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
-862841422647,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154241350,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
155323379,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
158699780,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [114]:
no_desc_df = pd.DataFrame(required_encoded.sum(axis=1).sort_values(), columns=['count']).query('count==0')
no_desc_df

,count
CID,
175606,0
12388,0
8141,0
-862841422647,0
11511,0
31285,0
11507,0
15600,0
12389,0


In [115]:
required_leffingwell_dataset_v2 = required_leffingwell_dataset.drop(no_desc_df.index).reset_index(drop=True)
required_leffingwell_dataset_v2.head()

,IsomericSMILES,Updated_Desc_v2,alcoholic,aldehydic,alliaceous,almond,amber,animal,anisic,apple,...,tropical,vanilla,vegetable,vetiver,violet,warm,waxy,weedy,winey,woody
0,CCCCC=COC(=O)CCCCCCCC,oily;fruity;waxy;herbal;green,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,CC(=O)OCC1C=CC(C(C)C)CC1,herbal;spicy;fruity;woody,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,CCCCCCCCC(OC(C)=O)C(=O)OC,peach;lactonic;apricot;buttery,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,CCCCC=COC(=O)C(C)CCC,tropical;green;fruity;apple,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
4,CCCCCCCC=CC(=O)OC(CCCCCCCC)C(=O)O,milky,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [116]:
required_leffingwell_dataset_v2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3510 entries, 0 to 3509
Columns: 139 entries, IsomericSMILES to woody
dtypes: int64(137), object(2)
memory usage: 3.7+ MB


In [117]:
required_encoded_v2 = required_leffingwell_dataset_v2.drop(columns=['IsomericSMILES', 'Updated_Desc_v2'])

In [118]:
# molecules with no descriptors
pd.DataFrame(required_encoded_v2.sum(axis=1).sort_values(), columns=['count']).query('count==0')

,count


In [119]:
list(required_encoded_v2.columns)

['alcoholic',
 'aldehydic',
 'alliaceous',
 'almond',
 'amber',
 'animal',
 'anisic',
 'apple',
 'apricot',
 'aromatic',
 'balsamic',
 'banana',
 'beefy',
 'bergamot',
 'berry',
 'bitter',
 'black currant',
 'brandy',
 'burnt',
 'buttery',
 'cabbage',
 'camphoreous',
 'caramellic',
 'cedar',
 'celery',
 'chamomile',
 'cheesy',
 'cherry',
 'chocolate',
 'cinnamon',
 'citrus',
 'clean',
 'clove',
 'cocoa',
 'coconut',
 'coffee',
 'cognac',
 'cooked',
 'cooling',
 'coumarinic',
 'creamy',
 'cucumber',
 'dairy',
 'dry',
 'earthy',
 'ethereal',
 'fatty',
 'fermented',
 'fishy',
 'floral',
 'fresh',
 'fruit skin',
 'fruity',
 'garlic',
 'gassy',
 'geranium',
 'grape',
 'grapefruit',
 'grassy',
 'green',
 'hawthorn',
 'hay',
 'hazelnut',
 'herbal',
 'honey',
 'hyacinth',
 'jasmin',
 'juicy',
 'ketonic',
 'lactonic',
 'lavender',
 'leafy',
 'leathery',
 'lemon',
 'lily',
 'malty',
 'meaty',
 'medicinal',
 'melon',
 'metallic',
 'milky',
 'mint',
 'muguet',
 'mushroom',
 'musk',
 'musty',
 'nat

In [120]:
required_encoded_v2.sum().sort_values(ascending=False)

fruity          1413
green            913
sweet            842
floral           558
fatty            412
                ... 
sandalwood         4
powdery            3
orangeflower       3
soapy              3
weedy              1
Length: 137, dtype: int64

In [121]:
required_leffingwell_dataset_v2[required_leffingwell_dataset_v2['IsomericSMILES'].duplicated(keep=False)]

,IsomericSMILES,Updated_Desc_v2,alcoholic,aldehydic,alliaceous,almond,amber,animal,anisic,apple,...,tropical,vanilla,vegetable,vetiver,violet,warm,waxy,weedy,winey,woody


In [122]:
required_leffingwell_dataset_v2[['IsomericSMILES', 'Updated_Desc_v2']][required_leffingwell_dataset_v2['IsomericSMILES']=='CC/C=C/CC(C)OC(=O)C']

,IsomericSMILES,Updated_Desc_v2
2731,CC/C=C/CC(C)OC(=O)C,green;fruity


In [123]:
required_leffingwell_dataset_v2.to_csv('curated_leffingwell.csv', index=False)